In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
filepath = "movies_metadata.csv"


In [3]:
# Create a dataframe from csv; drop all rows with no movie title.
df_raw = pd.read_csv(filepath, delimiter=',')
df = df_raw.dropna(subset=['title'])

# User list comprehension to create a list of lists from Dataframe rows
list_of_rows = [list(row) for row in df.values]
type(list_of_rows)

C:\Users\Walte\.conda\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


list

In [4]:
print(len(list_of_rows))

45460


In [5]:
#Check cell 3,3 to check 'genre' information..
print(list_of_rows[3][3])

[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]


In [6]:
#Check length of list - matches excel total number of movies.
print(len(list_of_rows))

45460


In [7]:
#String has to be converted into list of dictionaries
# found eval function on internet...seems to work.

jd = eval(list_of_rows[3][3])
print(jd)

[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]


In [8]:
#Try with another row..
jd2 = eval(list_of_rows[16][3])
print(jd2)

[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]


In [9]:
# add movie id number to what will be the genre dataframe.
df = pd.DataFrame(jd)
df['movie'] = list_of_rows[3][5]
df

,id,name,movie
0,35,Comedy,31357
1,18,Drama,31357
2,10749,Romance,31357


In [10]:
# Combine dataframes using the concat method..testing with just the two dataframes..

df2 = pd.DataFrame(jd2)
df2['movie'] = list_of_rows[4][5]
frames = [df,df2]
df3 = pd.concat(frames, sort=False)
df3

,id,name,movie
0,35,Comedy,31357
1,18,Drama,31357
2,10749,Romance,31357
0,18,Drama,11862
1,10749,Romance,11862


In [11]:
# Run a loop for all rows in the spreadsheet. 
#   convert the string in column[3] to python list of dictionaries
#   create new dataframe from genre information in each row
#   add movie id number from column[5] to dataframe
#   append to a list of dataframes to be concatenated outside the loop.
# Concatenate to one dataframe all the genre information with movie id. Reset index.

list_of_genre_dataframes = []
for row in list_of_rows:
    #print(row[3], row[5])
    genre_list = eval(row[3])
    record_df = pd.DataFrame(genre_list)
    record_df['movie_id_number'] = row[5]
    list_of_genre_dataframes.append(record_df)

genre_table = pd.concat(list_of_genre_dataframes, sort=False)
genre_table.reset_index(inplace=True, drop=True)
genre_table
    

,id,name,movie_id_number
0,16.0,Animation,862
1,35.0,Comedy,862
2,10751.0,Family,862
3,12.0,Adventure,8844
4,14.0,Fantasy,8844
...,...,...,...
91080,10751.0,Family,439050
91081,18.0,Drama,111109
91082,28.0,Action,67758
91083,18.0,Drama,67758


In [15]:
genre_df = genre_table[['id','name']]
genre_tbl_clean = genre_df.drop_duplicates()
genre_tbl_clean.rename(columns={'id':'genre_id','name':'genre'}, inplace=True)
genre_tbl_clean.reset_index()
genre_tbl_clean.head()

C:\Users\Walte\.conda\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,genre_id,genre
0,16.0,Animation
1,35.0,Comedy
2,10751.0,Family
3,12.0,Adventure
4,14.0,Fantasy


In [16]:
rds_connection_string = "postgres:postgres@localhost:5432/ETL"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

['genres',
 'keyword_movie_junction',
 'keywords',
 'ratings',
 'genre_movie_junction',
 'movies_data',
 'lang_movie_junction',
 'spoken_languages']

In [17]:
genre_tbl_clean.to_sql(name='genres', con=engine, if_exists='append', index=False)

In [18]:
genre_junction_df = genre_table[['id','movie_id_number']]
genre_junction_df.rename(columns={'id':'genre_id','movie_id_number':'movie_id'}, inplace=True)
genre_junction_df.reset_index()
genre_junction_df.head()

C:\Users\Walte\.conda\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,genre_id,movie_id
0,16.0,862
1,35.0,862
2,10751.0,862
3,12.0,8844
4,14.0,8844


In [19]:
genre_junction_df.to_sql(name='genre_movie_junction', con=engine, if_exists='append', index=False)